### 라이브러리 불러오기

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns 
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings(action='ignore')

In [2]:
import time
from glob import glob
from tqdm import tqdm
from scipy import interpolate

In [3]:
# conda install tensorflow

In [4]:
# conda install python=3.7

In [5]:
import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Activation, Dense, LSTM, GRU, AveragePooling1D, GlobalAveragePooling1D, Dropout, Flatten, BatchNormalization
from keras.layers.convolutional import Conv1D, MaxPooling1D, AveragePooling1D 
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras import backend as K

CNN으로 전처리해볼 것
1. 유량 3개, 수위 4개
2. 팔당댐 데이터

### 전체 수위 데이터셋 구축

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
# 데이터 불러오기 
전체df = pd.read_csv('/content/drive/MyDrive/신입기수 프로젝트/Dataset/rawdata.csv')
# for colab
# /content/drive/MyDrive/신입기수 프로젝트/Dataset/rawdata.csv
# for jupyter
# /Volumes/GoogleDrive/.shortcut-targets-by-id/1rUx7_7LmBqAQTbWEWVTGvw0_d0v6N2ZT/신입기수 프로젝트/Dataset/rawdata.csv

In [8]:
수위 = 전체df[['청담대교_수위', '한강대교_수위', '잠수교_수위', '행주대교_수위']]

In [9]:
수위

,청담대교_수위,한강대교_수위,잠수교_수위,행주대교_수위
0,310.7,290.0,300.2,275.3
1,314.7,290.0,300.2,275.3
2,313.7,290.0,301.2,275.3
3,311.7,290.0,301.2,276.3
4,311.7,291.0,301.2,277.3
...,...,...,...,...
276331,0.0,0.0,0.0,0.0
276332,0.0,0.0,0.0,0.0
276333,0.0,0.0,0.0,0.0
276334,0.0,0.0,0.0,0.0


In [10]:
nanIndex = 수위[수위.청담대교_수위.isnull()].index

In [11]:
nanIndex

Int64Index([140613, 140614, 140615, 140616, 140617, 140618, 140619, 140620,
            140621, 140622, 140623, 140624, 140625, 140626, 140627, 140628,
            140629, 140630, 140631, 140632, 140633, 140634, 140635, 140636,
            140637, 140638, 140639, 140640, 140641, 140642, 140643, 140644,
            140645, 140646, 140647, 140648, 140649, 140650, 140651, 140652,
            140653, 140654, 140655, 140656, 140657, 140658, 140659, 140660,
            140661, 140662, 140663, 140664, 140665, 140666, 140667, 140668,
            140669, 140670, 236080],
           dtype='int64')

In [12]:
consecutive = list(nanIndex)[:-1]
unconsecutive = list(nanIndex)[-1]
print(consecutive)
print(unconsecutive)

[140613, 140614, 140615, 140616, 140617, 140618, 140619, 140620, 140621, 140622, 140623, 140624, 140625, 140626, 140627, 140628, 140629, 140630, 140631, 140632, 140633, 140634, 140635, 140636, 140637, 140638, 140639, 140640, 140641, 140642, 140643, 140644, 140645, 140646, 140647, 140648, 140649, 140650, 140651, 140652, 140653, 140654, 140655, 140656, 140657, 140658, 140659, 140660, 140661, 140662, 140663, 140664, 140665, 140666, 140667, 140668, 140669, 140670]
236080


In [13]:
# 58개 결측치 앞 인덱스까지 인풋 생성
x = 수위.loc[:140612]

# 결측치로 아웃풋 생성
null = 수위[140613:140671]

In [14]:
null

,청담대교_수위,한강대교_수위,잠수교_수위,행주대교_수위
140613,NaN,NaN,NaN,NaN
140614,NaN,NaN,NaN,NaN
140615,NaN,NaN,NaN,NaN
140616,NaN,NaN,NaN,NaN
140617,NaN,NaN,NaN,NaN
140618,NaN,NaN,NaN,NaN
140619,NaN,NaN,NaN,NaN
140620,NaN,NaN,NaN,NaN
140621,NaN,NaN,NaN,NaN
140622,NaN,NaN,NaN,NaN


In [15]:
x

,청담대교_수위,한강대교_수위,잠수교_수위,행주대교_수위
0,310.7,290.0,300.2,275.3
1,314.7,290.0,300.2,275.3
2,313.7,290.0,301.2,275.3
3,311.7,290.0,301.2,276.3
4,311.7,291.0,301.2,277.3
...,...,...,...,...
140608,435.7,410.0,415.2,374.3
140609,438.7,413.0,418.2,375.3
140610,437.7,413.0,418.2,373.3
140611,434.7,413.0,415.2,373.3


In [16]:
# conda install scikit-learn

In [17]:
# train, test data 생성할 때 필요
from sklearn.model_selection import train_test_split
# 정규화할 때 필요
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler

In [18]:
# input 정규화
x_regular = x.copy()

for i in range(len(x.columns)):
    scaler = StandardScaler()                       #StandardScaler() , MinMaxScaler() , RobustScaler()
    a=np.array(x_regular.iloc[:,i])
    tmp_x = scaler.fit_transform(a.reshape(-1,1))
    x_regular.iloc[:,i] = tmp_x

In [19]:
x_regular

,청담대교_수위,한강대교_수위,잠수교_수위,행주대교_수위
0,-0.213183,-0.292141,-0.168958,-0.473340
1,-0.150481,-0.292141,-0.168958,-0.473340
2,-0.166156,-0.292141,-0.151149,-0.473340
3,-0.197507,-0.292141,-0.151149,-0.450834
4,-0.197507,-0.273710,-0.151149,-0.428328
...,...,...,...,...
140608,1.746252,1.919546,1.879052,1.754767
140609,1.793278,1.974838,1.932478,1.777274
140610,1.777603,1.974838,1.932478,1.732261
140611,1.730576,1.974838,1.879052,1.732261


### 한강대교 수위

In [20]:
start = time.time()

y_predict = []

xtrain = []
ytrain = []

w =  432 # window size
n = 58 # 예측할 데이터 개수

# xtrain 생성
# for i in range(len(x_regular)-w):
#     xtrain.append(np.array(x_regular.iloc[i:i + w, :]).astype(float))

for i in tqdm(range(len(x_regular)-w-n)):
    xtrain.append(np.array(x_regular.iloc[i:i + w, 1]).astype(float))

100%|██████████| 140123/140123 [00:15<00:00, 9041.46it/s]


In [21]:
# ytrain 생성
# for i in range(w, len(x_regular)):
#     ytrain.append(np.array(x_regular.iloc[i, :]).astype(float))
ytrain = []

for i in tqdm(range(w, len(x_regular)-n)):
  ytrain.append(np.array(x_regular.iloc[i:i+n, 1]).astype(float))

100%|██████████| 140123/140123 [00:17<00:00, 8235.40it/s]


In [22]:
xtrain = np.array(xtrain)
ytrain = np.array(ytrain)
print(xtrain.shape)
print(ytrain.shape)

(140123, 432)
(140123, 58)


In [23]:
df_xtrain = pd.DataFrame(xtrain, columns = [i for i in range(432)])

In [24]:
df_ytrain = pd.DataFrame(ytrain, columns = [f'R{i}' for i in range(58)])

In [25]:
df_series = pd.concat([df_xtrain, df_ytrain], axis = 1)

In [26]:
df_series.shape

(140123, 490)

In [27]:
xtrain.reshape(140123, 432, 1)
ytrain.reshape(140123, 58, 1)

array([[[-0.38429413],
        [-0.38429413],
        [-0.38429413],
        ...,
        [-0.12626405],
        [-0.16312549],
        [-0.19998693]],

       [[-0.38429413],
        [-0.38429413],
        [-0.38429413],
        ...,
        [-0.16312549],
        [-0.19998693],
        [-0.21841765]],

       [[-0.38429413],
        [-0.38429413],
        [-0.38429413],
        ...,
        [-0.19998693],
        [-0.21841765],
        [-0.23684837]],

       ...,

       [[ 0.09490458],
        [ 0.07647386],
        [ 0.05804314],
        ...,
        [ 1.84582295],
        [ 1.91954583],
        [ 1.97483799]],

       [[ 0.07647386],
        [ 0.05804314],
        [ 0.0211817 ],
        ...,
        [ 1.91954583],
        [ 1.97483799],
        [ 1.97483799]],

       [[ 0.05804314],
        [ 0.0211817 ],
        [ 0.00275098],
        ...,
        [ 1.97483799],
        [ 1.97483799],
        [ 1.97483799]]])

In [28]:
# label
labels_col = [f'R{i}' for i in range(58)]
labels = df_series[labels_col]
series = df_series.drop(labels_col, axis=1)
labels.values.shape

(140123, 58)

In [29]:
# x_train, x_val, y_train, y_val 분류
x_train, x_val, y_train, y_val = train_test_split(series, labels.values, test_size = 0.4, random_state = 0)

In [30]:
x_train

,0,1,2,3,4,5,6,7,8,9,...,422,423,424,425,426,427,428,429,430,431
133922,-0.365863,-0.365863,-0.365863,-0.384294,-0.384294,-0.402725,-0.439586,-0.439586,-0.458017,-0.458017,...,0.002751,0.058043,0.094905,0.094905,0.076474,0.021182,-0.034110,-0.070972,-0.089403,-0.126264
32870,-0.476448,-0.494878,-0.494878,-0.513309,-0.531740,-0.531740,-0.531740,-0.550171,-0.550171,-0.568601,...,-0.550171,-0.550171,-0.568601,-0.568601,-0.587032,-0.605463,-0.605463,-0.605463,-0.605463,-0.623893
53652,-0.402725,-0.384294,-0.365863,-0.384294,-0.402725,-0.421156,-0.458017,-0.494878,-0.476448,-0.458017,...,-0.808201,-0.808201,-0.808201,-0.808201,-0.808201,-0.808201,-0.808201,-0.808201,-0.808201,-0.808201
2311,-0.623893,-0.623893,-0.623893,-0.642324,-0.642324,-0.642324,-0.642324,-0.642324,-0.660755,-0.660755,...,-0.513309,-0.513309,-0.513309,-0.531740,-0.531740,-0.531740,-0.550171,-0.550171,-0.568601,-0.568601
132218,-0.605463,-0.605463,-0.623893,-0.623893,-0.642324,-0.642324,-0.642324,-0.642324,-0.660755,-0.660755,...,-0.421156,-0.439586,-0.458017,-0.494878,-0.513309,-0.531740,-0.531740,-0.550171,-0.550171,-0.568601
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41993,-0.015680,-0.015680,-0.015680,-0.015680,-0.015680,-0.015680,-0.015680,0.002751,0.002751,0.021182,...,0.223920,0.205489,0.223920,0.223920,0.168627,0.150197,0.150197,0.131766,0.113335,0.076474
97639,1.145456,1.071733,0.961148,0.868995,0.813703,0.758410,0.721549,0.666257,0.610965,0.537242,...,-0.955646,-0.955646,-0.955646,-0.955646,-0.955646,-0.955646,-0.955646,-0.955646,-0.955646,-0.955646
95939,-0.752909,-0.752909,-0.752909,-0.752909,-0.752909,-0.752909,-0.771339,-0.771339,-0.771339,-0.771339,...,-0.974077,-0.974077,-0.974077,-0.974077,-0.937216,-0.900354,-0.881924,-0.881924,-0.881924,-0.863493
117952,0.408227,0.371365,0.352935,0.334504,0.334504,0.316073,0.297642,0.279212,0.260781,0.242350,...,1.163886,1.182317,1.163886,1.108594,1.016441,0.942718,0.887426,0.868995,0.832133,0.795272


In [31]:
# xtest = 

In [32]:
# 모델링 및 모델 학습

if 'model' in locals(): 
    del model 
if 'history' in locals():
    del history


#He_normal
He_normal = keras.initializers.he_normal(seed=None)

model = Sequential()
model.add(Conv1D(filters = 8, input_shape = (w, 1), kernel_size = 2, padding='causal', dilation_rate=2, kernel_initializer = He_normal))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.2))  #0.2~0.5  0.8까지

model.add(Conv1D(filters = 16, kernel_size = 2, padding='causal', dilation_rate=4, kernel_initializer = He_normal))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.2))

model.add(Conv1D(filters = 32, kernel_size = 1))

model.add(Flatten())   #model.add(GlobalAveragePooling1D(pool_size=2))

model.add(Dense(50, kernel_initializer = He_normal))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.2))

model.add(Dense(58))


def root_mean_squared_error(y_true, y_pred):
        return K.sqrt(K.mean(K.square(y_pred - y_true))) 

opt = tf.keras.optimizers.Adam(lr=0.005) #기본값 0.001
model.compile(optimizer=opt, loss=root_mean_squared_error, metrics =["accuracy"])
model.summary()



callbacks = [keras.callbacks.EarlyStopping(monitor='val_loss',
                                            patience=5),
                keras.callbacks.ModelCheckpoint(filepath='best_model.h5',
                                                monitor='val_loss',
                                                save_best_only=True)]



history = model.fit(xtrain, ytrain, epochs=300, validation_data = (x_val, y_val), verbose=1, callbacks=callbacks, batch_size = 3742) #batch size..?


keras_model_best = keras.models.load_model('best_model.h5', custom_objects={'root_mean_squared_error': root_mean_squared_error})

keras_model_best.evaluate(x_val,y_val)

# y_predict.append(keras_model_best.predict(xtest))

print("time :", time.time() - start, "초")

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 432, 8)            24        
                                                                 
 batch_normalization (BatchN  (None, 432, 8)           32        
 ormalization)                                                   
                                                                 
 activation (Activation)     (None, 432, 8)            0         
                                                                 
 dropout (Dropout)           (None, 432, 8)            0         
                                                                 
 conv1d_1 (Conv1D)           (None, 432, 16)           272       
                                                                 
 batch_normalization_1 (Batc  (None, 432, 16)          64        
 hNormalization)                                        

In [33]:
x_test = x['한강대교_수위'][-432:]

In [35]:
x_test = np.array(x_test).reshape((1, 432, 1))

In [36]:
x_test.shape

(1, 432, 1)

In [37]:
y_predict = keras_model_best.predict(x_test)

In [38]:
y_predict.shape

(1, 58)

In [39]:
pd.Series(list(y_predict))

0    [336.10486, 336.3808, 336.21054, 335.4062, 335...
dtype: object

In [40]:
수위.loc[consecutive, '한강대교_수위'] = y_predict

In [41]:
수위.loc[consecutive, '한강대교_수위']

140613    336.104858
140614    336.380798
140615    336.210541
140616    335.406189
140617    335.918640
140618    334.322083
140619    333.930725
140620    332.997070
140621    331.616028
140622    331.178864
140623    330.283386
140624    328.946320
140625    328.630341
140626    326.996582
140627    326.707275
140628    325.258575
140629    324.944946
140630    322.517731
140631    321.849670
140632    319.713165
140633    318.045044
140634    314.717621
140635    315.585632
140636    312.909058
140637    308.405823
140638    307.188202
140639    305.688385
140640    303.813232
140641    302.326141
140642    299.850098
140643    298.738678
140644    296.559662
140645    295.746826
140646    295.561737
140647    296.294891
140648    295.091248
140649    293.718231
140650    293.154999
140651    291.302246
140652    291.085968
140653    290.053558
140654    288.007141
140655    288.058197
140656    285.401367
140657    283.728821
140658    283.254517
140659    281.277710
140660    279

In [42]:
# unconsecutive 부분 보간

unconsecutive2 = [unconsecutive-1, unconsecutive, unconsecutive+1]
수위.loc[unconsecutive2, '한강대교_수위'] = 수위.loc[unconsecutive2, '한강대교_수위'].interpolate()

In [43]:
수위['한강대교_수위'].isnull().sum()

0

In [44]:
수위['한강대교_수위'].to_csv('/content/drive/MyDrive/신입기수 프로젝트/Dataset/CNN_전처리/한강대교_수위.csv', index = False, encoding = 'cp949')